# Семинар 2

Casting, autograd и первая нейросеть!


## Casting

In [ ]:
# Helper to get what kind of tensor types
torch.*Tensor?

In [ ]:
m

tensor([[[2., 5., 3., 7.],
         [4., 2., 1., 9.]]])

In [ ]:
t = torch.tensor(m, dtype=torch.bfloat16, device='cuda:2')

<ipython-input-42-ad73a2903e09>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(m, dtype=torch.bfloat16, device='cuda')


In [ ]:
t.to(torch.float32)

tensor([[[2., 5., 3., 7.],
         [4., 2., 1., 9.]]], device='cuda:0')

In [ ]:
# This is basically a 64 bit float tensor
m_double = m.double()
m_double

In [ ]:
# This creates a tensor of type int8
m_byte = m.byte()
m_byte

In [ ]:
# Move your tensor to GPU device 0 if there is one (first GPU in the system)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
m.to(device)

tensor([[[2., 5., 3., 7.],
         [4., 2., 1., 9.]]], device='cuda:0')

In [ ]:
# Converts tensor to numpy array
m_np = m.numpy()
m_np

array([[2., 5., 3., 7.],
       [4., 2., 1., 9.]], dtype=float32)

In [ ]:
# In-place fill of column 0 and row 0 with value -1
m_np[0, 0] = -1
m_np

array([[-1.,  5.,  3.,  7.],
       [ 4.,  2.,  1.,  9.]], dtype=float32)

In [ ]:
m

tensor([[-1.,  5.,  3.,  7.],
        [ 4.,  2.,  1.,  9.]])

In [ ]:
# Create a tensor of integers ranging from 0 to 4
import numpy as np
n_np = np.arange(5)
n = torch.from_numpy(n_np)
print(n_np, n)

[0 1 2 3 4] tensor([0, 1, 2, 3, 4])


In [ ]:
# In-place multiplication of all elements by 2 for tensor n
# Because n is essentially n_np, not a clone, this affects n_np
n.mul_(2)


tensor([0, 2, 4, 6, 8])

In [ ]:
t = torch.Tensor(2, 2)

In [ ]:
t

tensor([[0.0000e+00, 0.0000e+00],
        [1.1351e-43, 0.0000e+00]])

In [ ]:
t[None, :, :]

tensor([[[0.0000e+00, 0.0000e+00],
         [1.1351e-43, 0.0000e+00]]])

## More fun

In [ ]:
# Creates two tensors of size 1x4
a = torch.Tensor([[1, 2, 3, 4]])
b = torch.Tensor([[5, 6, 7, 8]])
print(a.size(), b)

torch.Size([1, 4]) tensor([[5., 6., 7., 8.]])


In [ ]:
# Concatenate on axis 0, so you get 2x4
torch.cat((a, b), 0)

tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]])

In [ ]:
# Concatenate on axis 1, so you get 1x8
torch.cat((a, b), 1)

tensor([[1., 2., 3., 4., 5., 6., 7., 8.]])

### [Автоматическое дифференцирование](https://pytorch.org/docs/stable/notes/autograd.html)

In [ ]:
import torch
x = torch.rand(5)

x

tensor([0.0669, 0.1527, 0.4376, 0.7911, 0.3413])

In [ ]:
t = torch.tensor([2, 2, 3], dtype=torch.bfloat16, device='cuda', requires_grad=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
t.to('cuda')

tensor([2., 2., 3.], device='cuda:0', dtype=torch.bfloat16, requires_grad=True)

In [ ]:
w = torch.rand(3, 5, requires_grad=True)

w

tensor([[0.5917, 0.0359, 0.2763, 0.8524, 0.4772],
        [0.6546, 0.4645, 0.2239, 0.6609, 0.6132],
        [0.4657, 0.4555, 0.4197, 0.8770, 0.0949]], requires_grad=True)

In [ ]:
print(w.grad)

None


In [ ]:
first_z = torch.empty(3)

first_z

tensor([5.3021e-30, 0.0000e+00, 5.2559e-30])

In [ ]:
for i in range(3):
    first_z[i] = torch.sum(w[i] * x)

first_z

tensor([1.0032, 0.9448, 1.0106], grad_fn=<CopySlices>)

In [ ]:
z = torch.matmul(x, w.t())

z

tensor([1.0032, 0.9448, 1.0106], grad_fn=<SqueezeBackward4>)

In [ ]:
v = torch.rand(3, requires_grad=True)

v

tensor([0.4768, 0.8453, 0.3889], requires_grad=True)

In [ ]:
print(v.grad)

None


In [ ]:
y = torch.sum(z * v)

y

tensor(1.6700, grad_fn=<SumBackward0>)

In [ ]:
y.item()

1.669968605041504

In [ ]:
loss = torch.mean((y - 2) ** 2)

loss.item()

0.10892072319984436

In [ ]:
print(f'{x.grad=}\n')
print(f'{w.grad=}\n')
print(f'{z.grad=}\n')
print(f'{v.grad=}\n')

x.grad=None

w.grad=None

z.grad=None

v.grad=None



<ipython-input-24-b1828bd1c118>:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(f'{z.grad=}\n')


In [ ]:
loss.backward()

In [ ]:
print(f'{x.grad=}\n')
print(f'{w.grad=}\n')
print(f'{z.grad=}\n')
print(f'{v.grad=}\n')

x.grad=None

w.grad=tensor([[-0.0211, -0.0481, -0.1377, -0.2489, -0.1074],
        [-0.0373, -0.0852, -0.2442, -0.4414, -0.1904],
        [-0.0172, -0.0392, -0.1123, -0.2031, -0.0876]])

z.grad=None

v.grad=tensor([-0.6621, -0.6237, -0.6670])



<ipython-input-26-b1828bd1c118>:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(f'{z.grad=}\n')


## Bonus part

## Полносвязные слои и функции активации в `PyTorch`

In [ ]:
from torch import nn

### Полносвязный слой

>$y_j = \sum\limits_{i=1}^{n}x_iw_{ji} + b_j$


In [ ]:
nn.Linear?

In [ ]:
x

tensor([1.4548e-33, 0.0000e+00, 0.0000e+00, 2.3510e-38, 1.0489e-36])

In [ ]:
layer = nn.Linear(in_features=5, out_features=3)
x = torch.Tensor((5))
layer(x)


tensor([-5.0320e+24,  2.3017e+24, -3.2802e+24], grad_fn=<ViewBackward0>)

In [ ]:
layer.weight

Parameter containing:
tensor([[ 0.3524,  0.0397, -0.3125, -0.2296,  0.2872],
        [-0.1612,  0.2295,  0.3860,  0.1993, -0.0994],
        [ 0.2297, -0.1026,  0.3758,  0.3847,  0.2187]], requires_grad=True)

In [ ]:
display(layer.weight.shape)
display(layer.bias.shape)

torch.Size([3, 5])

torch.Size([3])

In [ ]:
layer = nn.Linear(in_features=5, out_features=3, bias=False)

In [ ]:
layer.weight.grad

### Функции активации

> Сигмоида $f(x) = \dfrac{1}{1 + e^{-x}}$

In [ ]:
activation = nn.Sigmoid()

In [ ]:
x = torch.randn(5)

print(x)

print(activation(x))

tensor([ 0.9211, -0.6740,  0.8187, -0.3292,  1.1349])
tensor([0.7153, 0.3376, 0.6940, 0.4184, 0.7567])


> ReLU $f(x) = \max(0, x)$

In [ ]:
activation = nn.ReLU()

In [ ]:
x = torch.randn(5)

print(x)

print(activation(x))

tensor([ 0.1228, -0.3128,  0.7083,  0.9356,  0.6187])
tensor([0.1228, 0.0000, 0.7083, 0.9356, 0.6187])


## Линейная регрессия

torch.Size([3, 1])

In [ ]:
import torch
from torch.autograd import Variable

x_data = Variable(torch.Tensor([[1.0], [2.0], [3.0]]))
y_data = Variable(torch.Tensor([[2.0], [4.0], [6.0]]))

display(x_data)
display(y_data)

class LinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()

        self.linear = torch.nn.Linear(1, 1, bias=False)  # One in and one out

    def forward(self, x):
        y_pred = self.linear(x)

        return y_pred


# our model
our_model = LinearRegressionModel()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(our_model.parameters(), lr = 0.01)

for epoch in range(100):
    # Forward pass: Compute predicted y by passing
    # x to the model
    pred_y = our_model(x_data)
    # Compute and print loss
    loss = criterion(y_data, pred_y)
    # Zero gradients, perform a backward pass,
    # and update the weights.
    # display(our_model.linear.weight.grad)
    optimizer.zero_grad()
    # display(our_model.linear.weight.grad)
    loss.backward()
    # display(our_model.linear.weight.grad)
    # display(our_model.linear.weight)
    optimizer.step()
    # display(our_model.linear.weight)
    print('epoch {}, loss {}'.format(epoch, loss.item()))
display(our_model.linear.weight)
new_var = Variable(torch.Tensor([[4.0]]))
pred_y = our_model(new_var)
print("predict (after training)", 4, our_model(new_var).item())


tensor([[1.],
        [2.],
        [3.]])

tensor([[2.],
        [4.],
        [6.]])

epoch 0, loss 19.167205810546875
epoch 1, loss 15.756294250488281
epoch 2, loss 12.952374458312988
epoch 3, loss 10.647427558898926
epoch 4, loss 8.752659797668457
epoch 5, loss 7.195074558258057
epoch 6, loss 5.914671421051025
epoch 7, loss 4.862123012542725
epoch 8, loss 3.9968807697296143
epoch 9, loss 3.285613775253296
epoch 10, loss 2.7009201049804688
epoch 11, loss 2.220276117324829
epoch 12, loss 1.8251657485961914
epoch 13, loss 1.500367283821106
epoch 14, loss 1.2333687543869019
epoch 15, loss 1.0138837099075317
epoch 16, loss 0.8334571719169617
epoch 17, loss 0.6851387023925781
epoch 18, loss 0.5632143020629883
epoch 19, loss 0.4629875123500824
epoch 20, loss 0.38059648871421814
epoch 21, loss 0.312867134809494
epoch 22, loss 0.2571905255317688
epoch 23, loss 0.21142220497131348
epoch 24, loss 0.1737985759973526
epoch 25, loss 0.14287002384662628
epoch 26, loss 0.1174454316496849
epoch 27, loss 0.09654537588357925
epoch 28, loss 0.07936457544565201
epoch 29, loss 0.0652411803

Parameter containing:
tensor([[1.9999]], requires_grad=True)

predict (after training) 4 7.99954891204834


In [ ]:
our_model.linear.bias

Parameter containing:
tensor([0.5274], requires_grad=True)

In [ ]:
1.7680*4 + our_model.linear.bias

tensor([7.5994], grad_fn=<AddBackward0>)

In [ ]:
torch.nn.Linear??